In [1]:
from transformers import AutoImageProcessor,  AutoTokenizer, AutoModelForCausalLM, ViTModel,  BertConfig, ViTConfig, VisionEncoderDecoderConfig, VisionEncoderDecoderModel, VisionEncoderDecoderModel ,ViTImageProcessor

from transformers import AutoImageProcessor,  AutoTokenizer, AutoModelForCausalLM, ViTModel,  BertConfig, ViTConfig, VisionEncoderDecoderConfig, VisionEncoderDecoderModel, VisionEncoderDecoderModel ,ViTImageProcessor




import torch







import torch






In [2]:
from transformers import AutoImageProcessor,  AutoTokenizer, AutoModelForCausalLM, ViTModel,  BertConfig, ViTConfig, VisionEncoderDecoderConfig, VisionEncoderDecoderModel, VisionEncoderDecoderModel ,ViTImageProcessor




In [3]:
configs = {
  "_name_or_path": "NinedayWang/PolyCoder-0.4B",
  "add_cross_attention": True,
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "bos_token_id": 0,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "rotary_emb_base": 10000,
  "rotary_pct": 1.0,
  "tie_word_embeddings": False,
  "tokenizer_class": "GPT2Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.27.3",
  "use_cache": True,
  "use_parallel_residual": False,
  "vocab_size": 50304
}

In [4]:
from DesignCoder.model import  GPTNeoXCrossAttentionModel


In [5]:



class GPTNeoXForCausalLM2(GPTNeoXPreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"position_ids", r"predictions.decoder.bias"]

    def __init__(self, config):
        super().__init__(config)

        self.gpt_neox = GPTNeoXModel(config)
        self.embed_out = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

        # Initialize weights and apply final processing
        self.post_init()
        
    def add_encoder(self, encoder):
        self.encoder = encoder 

    def get_output_embeddings(self):
        return self.embed_out

    def set_output_embeddings(self, new_embeddings):
        self.embed_out = new_embeddings

    def forward(
            self,
            input_ids: Optional[torch.LongTensor] = None,
            attention_mask: Optional[torch.FloatTensor] = None,
            inputs_embeds: Optional[torch.FloatTensor] = None,
            head_mask: Optional[torch.FloatTensor] = None,
            past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
            labels: Optional[torch.LongTensor] = None,
            use_cache: Optional[bool] = None,
            output_attentions: Optional[bool] = None,
            output_hidden_states: Optional[bool] = None,
            return_dict: Optional[bool] = None,
            encoder_attention_mask=None,
            encoder_hidden_states=None,
        ) -> Union[Tuple, CausalLMOutputWithPast]:
        r"""
        past_key_values (`tuple(tuple(torch.FloatTensor))`, *optional*, returned when `use_cache=True` is passed or when `config.use_cache=True`):
            Tuple of `tuple(torch.FloatTensor)` of length `config.n_layers`, with each tuple having 2 tensors of shape
            `(batch_size, num_heads, sequence_length, embed_size_per_head)`) and 2 additional tensors of shape
            `(batch_size, num_heads, encoder_sequence_length, embed_size_per_head)`. The two additional tensors are
            only required when the model is used as a decoder in a Sequence to Sequence model.
            Contains pre-computed hidden-states (key and values in the self-attention blocks that can be used (see
            `past_key_values` input) to speed up sequential decoding.
            If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
            don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
            `decoder_input_ids` of shape `(batch_size, sequence_length)`.
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the left-to-right language modeling loss (next word prediction). Indices should be in
            `[-100, 0, ..., config.vocab_size]` (see `input_ids` docstring) Tokens with indices set to `-100` are
            ignored (masked), the loss is only computed for the tokens with labels n `[0, ..., config.vocab_size]`.
        use_cache (`bool`, *optional*):
            If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
            `past_key_values`).
        Returns:
        Example:
        ```python
        >>> from transformers import AutoTokenizer, GPTNeoXForCausalLM, GPTNeoXConfig
        >>> import torch
        >>> tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
        >>> config = GPTNeoXConfig.from_pretrained("EleutherAI/gpt-neox-20b")
        >>> config.is_decoder = True
        >>> model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/gpt-neox-20b", config=config)
        >>> inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
        >>> outputs = model(**inputs)
        >>> prediction_logits = outputs.logits
        ```"""
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        import pdb; pdb.set_trace()
        outputs = self.gpt_neox(
            input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_states = outputs[0]
        lm_logits = self.embed_out(hidden_states)

        lm_loss = None
        if labels is not None:
            # we are doing next-token prediction; shift prediction scores and input ids by one
            shift_logits = lm_logits[:, :-1, :].contiguous()
            labels = labels[:, 1:].contiguous()
            loss_fct = CrossEntropyLoss()
            lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), labels.view(-1))

        if not return_dict:
            output = (lm_logits,) + outputs[1:]
            return ((lm_loss,) + output) if lm_loss is not None else output

        return CausalLMOutputWithPast(
            loss=lm_loss,
            logits=lm_logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
    
    
    def cross_attention_adder(self, ):
        
        self.enc_to_dec_proj = nn.Linear(self.encoder.config.hidden_size, self.decoder.config.hidden_size)

        
        if self.encoder.get_output_embeddings() is not None:
            raise ValueError(
                f"The encoder {self.encoder} should not have a LM Head. Please use a model without LM Head"
            )
        # check tieing.
    
    def encoder_decoder_forward(input_ids):
        
        self.encoder(input_ids)
        
        return self.forward()

    def prepare_inputs_for_generation(self, input_ids, past_key_values=None, attention_mask=None, **model_kwargs):
        input_shape = input_ids.shape

        # if model is used as a decoder in encoder-decoder model, the decoder attention mask is created on the fly
        if attention_mask is None:
            attention_mask = input_ids.new_ones(input_shape)

        # cut decoder_input_ids if past is used
        if past_key_values and past_key_values[0] is not None:
            input_ids = input_ids[:, -1:]

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "past_key_values": past_key_values,
        }

    def _reorder_cache(self, past_key_values, beam_idx):
        reordered_past = ()
        for layer_past in past_key_values:
            reordered_past += (
                tuple(past_state.index_select(0, beam_idx) for past_state in layer_past[:2]) + layer_past[2:],
            )
        return reordered_past

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from packaging import version
assert version.parse(transformers.__version__) >= version.parse("4.23.0")


"""model = GPTNeoXForCausalLM.from_pretrained("Code-LMs/Convert2HF/polycoder/0-4B/")
tokenizer = AutoTokenizer.from_pretrained("NinedayWang/PolyCoder-2.7B")
"""

model_path = "/Users/mehmetburaksayici/Desktop/sahibinden/ccrraawwll/_designcoder/"
tokenizer_path = "/Users/mehmetburaksayici/Desktop/sahibinden/ccrraawwll/_designcoder_tokenizer/"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

decoder = GPTNeoXForCausalLM2.from_pretrained(model_path)


In [12]:
prompt = '''def binarySearch(arr, left, right, x):
    mid = (left +'''
input_ids = tokenizer.encode(prompt, return_tensors='pt')
result = decoder.generate(input_ids, max_length=50, num_beams=1, num_return_sequences=1)
for res in result:
    print(tokenizer.decode(res))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


> /var/folders/96/vgb_33jj74d617ss5z5jjvr40000gn/T/ipykernel_9382/772052102.py(67)forward()
     65         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     66         import pdb; pdb.set_trace()
---> 67         outputs = self.gpt_neox(
     68             input_ids,
     69             attention_mask=attention_mask,

ipdb> c
> /var/folders/96/vgb_33jj74d617ss5z5jjvr40000gn/T/ipykernel_9382/772052102.py(67)forward()
     65         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     66         import pdb; pdb.set_trace()
---> 67         outputs = self.gpt_neox(
     68             input_ids,
     69             attention_mask=attention_mask,

ipdb> c
> /var/folders/96/vgb_33jj74d617ss5z5jjvr40000gn/T/ipykernel_9382/772052102.py(67)forward()
     65         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     66         import pdb; pdb.set_trace()
---> 67         output

ipdb> c
> /var/folders/96/vgb_33jj74d617ss5z5jjvr40000gn/T/ipykernel_9382/772052102.py(67)forward()
     65         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     66         import pdb; pdb.set_trace()
---> 67         outputs = self.gpt_neox(
     68             input_ids,
     69             attention_mask=attention_mask,

ipdb> c
> /var/folders/96/vgb_33jj74d617ss5z5jjvr40000gn/T/ipykernel_9382/772052102.py(67)forward()
     65         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     66         import pdb; pdb.set_trace()
---> 67         outputs = self.gpt_neox(
     68             input_ids,
     69             attention_mask=attention_mask,

ipdb> c
> /var/folders/96/vgb_33jj74d617ss5z5jjvr40000gn/T/ipykernel_9382/772052102.py(67)forward()
     65         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     66         import pdb; pdb.set_trace()
---> 67       

ipdb> c
> /var/folders/96/vgb_33jj74d617ss5z5jjvr40000gn/T/ipykernel_9382/772052102.py(67)forward()
     65         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     66         import pdb; pdb.set_trace()
---> 67         outputs = self.gpt_neox(
     68             input_ids,
     69             attention_mask=attention_mask,

ipdb> c
> /var/folders/96/vgb_33jj74d617ss5z5jjvr40000gn/T/ipykernel_9382/772052102.py(67)forward()
     65         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     66         import pdb; pdb.set_trace()
---> 67         outputs = self.gpt_neox(
     68             input_ids,
     69             attention_mask=attention_mask,

ipdb> c
> /var/folders/96/vgb_33jj74d617ss5z5jjvr40000gn/T/ipykernel_9382/772052102.py(67)forward()
     65         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     66         import pdb; pdb.set_trace()
---> 67       

In [13]:

#tokenizer = AutoTokenizer.from_pretrained("NinedayWang/PolyCoder-0.4B")
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

tokenizer = AutoTokenizer.from_pretrained("NinedayWang/PolyCoder-0.4B")


encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

encoder_config = encoder.config
encoder_config.add_cross_attention = True
encoder = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k",config = encoder_config)


decoder 





# Initializing a model with a pretrained Swin as encoder & a pretrained BART-large as decoder
model = VisionEncoderDecoderModel(encoder=encoder, decoder=decoder)






In [39]:
encoder()

NameError: name 'self' is not defined

In [16]:
from typing import * 
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import io, transforms
import torchvision.datasets as dset
from tqdm.auto import tqdm
from transformers import EncoderDecoderModel, GPT2Tokenizer, ViTFeatureExtractor


MEAN = (0.485, 0.456, 0.406)
STD = (0.229, 0.224, 0.225)

TRAIN_PCT = 0.95
NUM_WORKERS = 2
BATCH_SIZE = 8
EPOCHS = 3
LR = 1e-4
IMAGE_SIZE = (224, 224)

MAX_TEXT_LENGTH = 32

LABEL_MASK = -100

TOP_K = 1000
TOP_P = 0.95


tfms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE), 
        transforms.ToTensor(),
        transforms.Normalize(
            mean=0.5, 
            std=0.5
        )
   ]
)
descale = transforms.Compose(
    [
        transforms.Normalize(
            mean = [ 0., 0., 0. ],
            std = 1 / 0.5
        ),
        transforms.Normalize(
            mean = -0.5,
            std = [ 1., 1., 1. ]
        ),                           
    ]
)

In [37]:


images =  Image.fromarray(np.zeros((224,224,3)).astype(np.uint8))

pixel_values = image_processor(images).pixel_values
inp_ = torch.from_numpy(pixel_values[0])[None,]
encoder_outputs = encoder(inp_)


In [38]:

model(inp_)

> /var/folders/96/vgb_33jj74d617ss5z5jjvr40000gn/T/ipykernel_9382/772052102.py(67)forward()
     65         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
     66         import pdb; pdb.set_trace()
---> 67         outputs = self.gpt_neox(
     68             input_ids,
     69             attention_mask=attention_mask,

ipdb> input_ids
ipdb> input_ids
ipdb> input_ids
ipdb> c


ValueError: You have to specify either input_ids or inputs_embeds

In [43]:
decoder.gpt_neox.layers[0]

GPTNeoXLayer(
  (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (attention): GPTNeoXAttention(
    (rotary_emb): RotaryEmbedding()
    (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
    (dense): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (mlp): GPTNeoXMLP(
    (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
    (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
    (act): GELUActivation()
  )
)

In [ ]:

"""

def generate_sentence_from_image(model, encoder_outputs, tokenizer, max_text_length: int, device)-> List[str]:
    generated_so_far = torch.LongTensor([[tokenizer.bos_token_id]]*len(encoder_outputs.last_hidden_state)).to(device)
    with torch.no_grad():
        for _ in tqdm(range(max_text_length)):
            attention_mask = torch.ones_like(generated_so_far)
            decoder_out = model(
                decoder_input_ids=generated_so_far, 
                decoder_attention_mask=attention_mask,
                encoder_outputs=encoder_outputs
            )

            next_token_logits = decoder_out["logits"][:, -1, :]
            filtered_p = top_k_top_p_filtering(next_token_logits, top_k=TOP_K, top_p=TOP_P, device=device)
            next_token = torch.multinomial(filtered_p, num_samples=1)
            generated_so_far = torch.cat((generated_so_far, next_token), dim=1)

    return [tokenizer.decode(coded_sentence) for coded_sentence in generated_so_far]




images =  Image.fromarray(np.zeros((224,224,3)).astype(np.uint8))

pixel_values = image_processor(images).pixel_values

encoder_outputs = model.encoder(pixel_values=tfms(images)[None,:])



generated_sentences = generate_sentence_from_image(
                  model, 
                encoder_outputs, 
                tokenizer, 
                MAX_TEXT_LENGTH,
                "cpu"
            )"""